In [ ]:
pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org  --upgrade pip

In [ ]:
!pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org  \
pinecone-client \
pandas \
langchain \
sentence-transformers

In [10]:
from pinecone import Pinecone
import os
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
from langchain.document_loaders import TextLoader
from langchain.document_loaders import  DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

ModuleNotFoundError: No module named 'torch._C'

In [ ]:
# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
environment = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

# configure client
pc = Pinecone(api_key=api_key)

# Prepare Data - New Articles

In [ ]:
# load multiple document and process documants
loader = DirectoryLoader("./bbc/sport", glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()

In [ ]:
# split the text into smaler chunks
text_spliter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_spliter.split_documents(documents)

# Cesate Embeddings and Vectors

In [ ]:
# average_word_embeddings_komninos
# dimension = 300
model = SentenceTransformer('average_word_embeddings_komninos')

In [ ]:
# check dimensionality of embeddings

# Setupup Pinecone index
	- Make sure the dimenionality of the embeddings matches the dimensionality of the index

# Upsert vector data

# Query vector data

# Query with metadata filters

# Cleanup